# 演習
- モデルをつくって、物質量や速度定数をさまざまに設定して、それぞれの定常状態を検討してみよう。

## E-Cell3を使うための準備

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

## %install_ext https://raw.githubusercontent.com/naito/ecell3-ipython/master/extensions/ecell3.py
## %load_ext ecell3

In [ ]:
import biosim_course

# モデル１

![モデル１概要](./model2.png)

# このモデルに含まれる `Entity`

## `System`
- ルートシステム（細胞外の空間） FullID = `System::/'
- 細胞 FullID = `System:/:Cell'

## `Variable`
- 物質１ (`S1`)　 FullID = `Varible:/Cell:S1'
- 物質２ (`S2`)　 FullID = `Varible:/Cell:S2'
- 物質２ (`S3`)　 FullID = `Varible:/Cell:S3'


## `Process`
- 物質１ (`E1`)　 FullID = `Process:/Cell:E1'
- 物質２ (`E2`)　 FullID = `Process:/Cell:E2'

# モデルをつくる

In [ ]:
EM = '''
Stepper ODEStepper( ODE ){}

System System( / )
{
    StepperID    ODE;

    Variable Variable( SIZE )
    {
        Value    1;
    }
}

System System( /Cell )
{
    StepperID    ODE;

Variable Variable( SIZE )
    {
        Value    1;
    }









}
'''

# モデルを走らせてみる

In [ ]:
setModel(EM, "model")

print 't = {}'.format( getCurrentTime() )

## Loggerをつくる

In [ ]:
target_SystemPath_list = ( '/', '/Cell', )
Target_Properties = { 'Variable': ['Value', 'Velocity'], }
Logger_dict = {}

for target_SystemPath in target_SystemPath_list:
    for E_type, Properties in Target_Properties.items():
        for E in getEntityList( E_type, target_SystemPath ):
            for p in Properties:
                FullPN = ':'.join( ( E_type, target_SystemPath, E, p ) )
                Logger_dict[ FullPN ] = createLoggerStub( FullPN )
                Logger_dict[ FullPN ].create()

## シミュレーションを実行する

In [ ]:
step_width = 60.0  # min

run( step_width )
print 't = {}'.format( getCurrentTime() )

## グラフを描いてみる

In [ ]:
Data_dict = {}
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( 0, getCurrentTime(), getCurrentTime() / 100 ) )[ :, :2 ]

FullPNs_for_plot = [ 
    'Variable:/Cell:S1:Value',
    'Variable:/Cell:S2:Value',
    'Variable:/Cell:S3:Value',
    ]

plt.figure()
for FullPN, d in Data_dict.items():
    if FullPN in FullPNs_for_plot:
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )